# FIN 514 - PS3 Q1 - Airbag Autocallable

**Spring 2023**

This notebook provides Python codes for the valuation in PS3 Q1

## Packages and Configurations

The following common packages will be use on this notebook.

* numpy - [https://numpy.org/](https://numpy.org/)
* Pandas - [https://pandas.pydata.org/](https://pandas.pydata.org/)
* matplotlib - [https://matplotlib.org/](https://matplotlib.org/)
* Scipy Statistical functions - [https://docs.scipy.org/doc/scipy/reference/stats.html](https://docs.scipy.org/doc/scipy/reference/stats.html)


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

### Parameter Set up

In [2]:
# ENTER INPUT FOR: N = num_steps

N = 100

In [3]:
# ENTER INPUT FOR: S0 = Original Stock Price

S0 = 227.29

In [4]:
# ENTER INPUT FOR: sigma = Annualized (Future) Volatility of Stock Price Returns

sigma = 0.3573

In [5]:
# ENTER INPUT FOR: r = Annualized Continously Compounded Risk-free Rate

r = 0.048

In [6]:
# ENTER INPUT FOR: T = Option maturity

T = 367/365

In [7]:
# ENTER INPUT FOR: DIV = Size of proportional dividend
# ND as number of dividend dates
DIV = 0.0219/4
ND = 4

In [8]:
#Enter INPUT FOR: Dates when dividends are paid, here assuming the equivalent dates as in 2020. That is 
TD = np.zeros([ND])
TD = [41/365, 136/365, 221/365, 315/365]


In [9]:
# ENTER INPUT FOR: whether option is call (1) or put (0)
cp = 0

### Stock price tree

In [10]:
def Stock_tree(N, S0, sigma, r, T, DIV, TD):
    
    stock_value = np.zeros([N+1, N+1])    
    
    delta = T / N
    u = np.exp(r*delta + sigma * (delta)**0.5)
    d = np.exp(r*delta - sigma * (delta)**0.5)
    print("u = ", u, "d=", d)

    # FIRST LET'S BUILD A STOCK PRICE TREE WITH DIVIDENDS  
    # Let's have dividends at grid points rather than times
    jD1 = [i/delta for i in TD]
    #print(jD1)
    jD = [np.ceil(i) for i in jD1]
    print(jD)
    
    stock_value[0,0] = S0
    #print(stock_value[0,0])
    for j in range (1,N+1):
        stock_value[j, 0] = stock_value[j-1, 0]*d 
        for i in range(1, j+1):    
            stock_value[j, i] = stock_value[j-1, i-1]*u      

    # This adjusts all stock prices for that j by the size of the dividend    
        if j in jD: stock_value[j, :] *= (1-DIV)
    print(stock_value[3,0], stock_value[4,0], stock_value[5,0])        
    return stock_value

In [11]:
stock = Stock_tree(N, S0, sigma, r, T, DIV, TD)



u =  1.0369776607409562 d= 0.9652722176988613
[12.0, 38.0, 61.0, 86.0]
204.42299572308679 197.3238384302688 190.47121912643738


In [12]:
print(stock)

[[2.27290000e+02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.19396722e+02 2.35694653e+02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.11777561e+02 2.27509500e+02 2.44410089e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [6.96184248e+00 7.47900436e+00 8.03458372e+00 ... 7.80628512e+03
  0.00000000e+00 0.00000000e+00]
 [6.72007313e+00 7.21927513e+00 7.75556045e+00 ... 7.53519015e+03
  8.09494328e+03 0.00000000e+00]
 [6.48669989e+00 6.96856571e+00 7.48622703e+00 ... 7.27350970e+03
  7.81382385e+03 8.39427535e+03]]


### Airbag autocallable valuation 

In [13]:
#key details
Face = 1000
ratio = 1000/(0.85*S0)

In [14]:
#Size of coupon and number of coupons per year
cpn = 0.1185
nc = 12

In [15]:
#Coupon only dates
tco = [33/365, 66/365, 124/365, 153/365, 215/365, 245/365, 307/365, 339/365]

In [16]:
#autocall observation dates
tac = [90/365, 182/365, 276/365]

In [17]:
#autocall payment dates
tacp = [94/365, 186/365, 278/365]

In [18]:

def Bin_autocall(N, S0, Face, ratio, sigma, r, T, stock_value, cpn, nc, tco, tac, tacp):

    # LIST TO SAVE RESULTS
    autocall_result = []
        
    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [N+1,N+1] TO STORE ALL STEPS
    # option_value[N+1, N+1]
    option_value = np.zeros([N+2, N+2])
    
    delta = T / N
    u = np.exp(r*delta+sigma * (delta)**0.5)
    d = np.exp(r*delta-sigma * (delta)**0.5)
    q = (np.exp(r * delta) - d) / (u - d)

    #First, let's calculate the coupon dates and accrued interest

    #jC1 are the exact values of j where the coupons are paid
    #jC are the values of j where we first factor in the coupons - 
    #where int will give us the j immediately before or on the call date
    jco1 = [j/delta for j in tco]
    jco = [int(j) for j in jco1]
    
    jac1 = [j/delta for j in tac]
    jac = [int(j) for j in jac1]
    #This converts the call times into times steps, 
    #where int will give us the j immediately before the call date
    
    j = N
    for i in range(0, j+1): 
        option_value[j, i] = (ratio*stock_value[j,i]+Face*cpn/nc)*np.exp(-r*3/365) 
        if stock_value[j,i] > 0.85*S0: option_value[j, i] = Face*(1+cpn/nc)*np.exp(-r*3/365) 
        
        
    for j in range(N-1, -1, -1):
        if j in jco: print("coupon only",j,jco.index(j))
        if j in jac: print("autocall",j,jac.index(j))    
        for i in range(0, j+1):           
            cont = np.exp(-r * delta) * (q * option_value[j + 1, i + 1] + (1-q) * option_value[j + 1, i])
            if j in jco: 
                cont = cont + Face*cpn/nc*np.exp(-r*(tco[jco.index(j)]-delta*j))  
            # the "if j in jC" is a really great Python command as it will check whether the entry ever occurs 
            # in the array jC. The index command is a little erratic but here will find the equivalent entry in 
            # the tc array - again a feature that may be helpful in future projects.
            if j in jac:
                if stock_value[j,i]>=S0: 
                    cont = Face*(1+cpn/nc)*np.exp(-r*(tacp[jac.index(j)]-delta*j))
                else:
                    cont = cont + Face*cpn/nc*np.exp(-r*(tacp[jac.index(j)]-delta*j))
            option_value[j,i] = cont
            
      
    output = {'num_steps': N, 'Value': option_value[0,0]}
    autocall_result.append(output)

    return autocall_result


In [19]:
Autocall = Bin_autocall(N, S0, Face, ratio, sigma, r, T, stock, cpn, nc, tco, tac, tacp)

coupon only 92 7
coupon only 83 6
autocall 75 2
coupon only 66 5
coupon only 58 4
autocall 49 1
coupon only 41 3
coupon only 33 2
autocall 24 0
coupon only 17 1
coupon only 8 0


In [20]:
# CREATE A DATAFRAME FROM THE BINOMIAL MODEL OUTPUT
df = pd.DataFrame.from_dict(Autocall)

In [21]:
# INSPECT THE FIRST ROWS OF THE DATAFRAME
df.head()

,num_steps,Value
0,100,986.456313
